In [1]:
import pandas as pd

# Path to my file – it can be any file
file_path = "C:/Users/DELL/Desktop/deliveries.csv"

# Load the CSV file with the correct separator
df = pd.read_csv(file_path, sep=";", encoding="utf-8", engine="python")

# Settings to view all columns and full cell content, since after downloading the CSV it looked strange and I needed to make sure everything was in order and in place
pd.set_option('display.max_columns', None)        # Show all columns
pd.set_option('display.max_colwidth', None)       # Show full content of each cell
pd.set_option('display.width', None)              # Adjust width
pd.set_option('display.expand_frame_repr', False) # Don't split columns into multiple lines

# Show the first rows of the dataframe
df.head()


,id,campaign_id,campaign_name,newsletter_id,newsletter_name,template_id,template_name,transactional_message_id,transactional_message_name,subject,action_id,parent_action_id,customer_id,recipient,created,created_RFC3339,failure_message,sent,sent_RFC3339,delivered,delivered_RFC3339,opened,opened_RFC3339,clicked,clicked_RFC3339,converted,converted_RFC3339,bounced,bounced_RFC3339,spammed,spammed_RFC3339,unsubscribed,unsubscribed_RFC3339,suppressed,suppressed_RFC3339,failed,failed_RFC3339,drafted,drafted_RFC3339,topic_unsubscribed,topic_unsubscribed_RFC3339,email
0,RPCtBwUAAYw2N2cGLMlm-0X9RlZ8ZQ==,NaN,NaN,NaN,NaN,242,Low data,4.0,Low data,Running out of data?,NaN,NaN,dfbe3f4a5782a97646a6ce9cc91b234a,reggidori@allin-partners.com,1701716649,4/12/2023 14:04,"No MX for allin-partners.com, lookup allin-partners.com on 172.17.0.1:53: no such host",1701716655,4/12/2023 14:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1701716658,4/12/2023 14:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,reggidori@allin-partners.com
1,RPCtBwUAAYw2JZ5ZnHbPWa9Mgfeamw==,NaN,NaN,NaN,NaN,244,Video uploaded,6.0,Video uploaded,Video successfully uploaded!,NaN,NaN,bb4f7c2e1397449ff338881f23de389d,hamdi@videoblogg.com,1701715484,4/12/2023 13:44,5.1.1 The email account that you tried to reach does not exist. Please try\n5.1.1 double-checking the recipient's email address for typos or\n5.1.1 unnecessary spaces. Learn more at\n5.1.1 https://support.google.com/mail/?p=NoSuchUser n8-20020a0cfbc8000000b0067a1fc7914bsi8523477qvp.236 - gsmtp,1701715486,4/12/2023 13:44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1701715487,4/12/2023 13:44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hamdi@videoblogg.com
2,dgTwrQcAAMfGCMbGCAGMNcXPsI8buTfq_C49eIw=,28.0,Signup onboarding v3,NaN,NaN,223,Share live stream features,NaN,NaN,Take your streams to the next level with Dacast,419.0,NaN,eac2145c-eb90-a425-2793-16d360ca8172,maheshmahi@gamil.con,1701709205,4/12/2023 12:00,"No MX for gamil.con, lookup gamil.con on 172.17.0.1:53: no such host",1701709209,4/12/2023 12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1701709213,4/12/2023 12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,maheshmahi@gamil.con
3,dgTwrQcAALjJCLfJCAGMNcXPpfPDnufKi-jyyz0=,28.0,Signup onboarding v3,NaN,NaN,205,Share live stream features,NaN,NaN,Take your streams to the next level with Dacast,372.0,NaN,f9a98d62-5945-adff-2229-8359d98ac536,anshul@realimapct.tv,1701709205,4/12/2023 12:00,"No MX for realimapct.tv, lookup realimapct.tv on 172.17.0.1:53: no such host",1701709207,4/12/2023 12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1701709209,4/12/2023 12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,anshul@realimapct.tv
4,dgTwrQcAAJnJCJjJCAGMNcXPpe2eqZyz7iMBYxs=,28.0,Signup onboarding v3,NaN,NaN,222,Onboarding Stream,NaN,NaN,Start streaming with Dacast today,418.0,NaN,1a506fdc-c117-8b76-7a5a-57aab8571700,johncookj11@netzero.com,1701709205,4/12/2023 12:00,temporary delay due to high volume,1701709207,4/12/2023 12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1701709216,4/12/2023 12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,johncookj11@netzero.com


In [2]:
# Show all rows without ellipses, to see each possible value in a clear and simple way
pd.set_option('display.max_rows', None)

# Extract the unique non-null values as a table, to make it as understandable as possible
unique_failures_df = pd.DataFrame(df['failure_message'].dropna().unique(), columns=['failure_message'])

# Display the table in order of appearance
unique_failures_df = unique_failures_df.sort_values(by='failure_message').reset_index(drop=True)



In [3]:
# https://senderscore.org/assess/bounce-lookup/email-error/5-X-X/, JUST CHANGE THE NUMBERS FOR THE ERROR YOU MAY WANT TO SEE

# Filter the unique non-null values from the 'failure_message' column, to see all possible email errors
unique_failures_df = pd.DataFrame(df['failure_message'].dropna().unique(), columns=['failure_message'])

# Sort by message and format to streamline the information
unique_failures_df = unique_failures_df.sort_values(by='failure_message').reset_index(drop=True)

# Save the original total, just for later reference
original_count = len(unique_failures_df)

# Keywords to identify non-existent user messages

# Due to ambiguity, I will declare that NON-FUNCTIONAL ≠ NON-EXISTENT

# Keywords that clearly indicate that the email does not exist. I know I could use more specific terms to avoid generalizing,
# but for the CSV case it’s not necessary, so I’m keeping it broad. For example, the first one could be "The email account that you tried to reach does not exist"
# I also know some overlap, but better safe than sorry

sure_keywords = [
    "does not exist",
    "5.1.1",
    "user doesn't exist",
    "no such host",
    "user unknown",
    "domain accepts no mail",
    "invalid recipient",
    "mailbox not found"
]

# COMMENTS ON EACH PATTERN THAT COULD CAUSE SUSPICION OR I AM SURE SHOULD BE USED TO FILTER THESE EMAILS

# "does not exist" - Indicates the user does not exist in the system or domain. This even applies to "mailbox does not exist!"
# "user doesn't exist" - Similar to the above, indicates that the user cannot be found, but needs the apostrophe.
# "no such host" - Error related to a domain or host that cannot be found on the network; the email doesn’t exist in that network.
# "user unknown" - Similar to the first case, suggests the user is not recognized by the system.
# "domain accepts no mail" - The domain does not accept emails, implying the destination is invalid.
# "5.7.13" - SMTP error code indicating the server has rejected the message, possibly due to a user-related issue.
# "550 5.0.1" - SMTP error code that typically means the email wasn't delivered because the user doesn't exist.
# "5.1.1" - Common SMTP error indicating that the email address is invalid or does not exist.
# "invalid recipient" - Indicates that the email recipient is not valid and therefore does not exist.
# "sender verify failed" - Error that occurs when sender verification fails, typically associated with non-existent recipients.
# "mailbox is disabled" - Indicates the mailbox is disabled but exists, and could be reactivated.
# "i/o timeout" could also be considered, BUT there is no certainty about whether the user exists or not.
# 5.4.1 documentation declares it as a busy network, with no indication of non-existence.
# 5.7.13 error means that it exists but is currently deactivated or inactive.
# 5.2.1 means the account is inactive and cannot receive messages UNTIL reactivated, but it exists and could be functional.
# "mailbox unavailable" really only means it cannot be accessed at the moment.

# Keywords that *suggest* the email does not exist, since many of them are possible indicators, but not 100% certain
suspicious_keywords = [
    "550 5.0.1",
    "sender verify failed"
]

non_existent_keywords = sure_keywords + suspicious_keywords

def contains_non_existent(msg):
    return any(keyword in msg.lower() for keyword in non_existent_keywords)

# Create DataFrame with messages that indicate non-existent users
no_existent_df = unique_failures_df[unique_failures_df['failure_message'].apply(contains_non_existent)]

# Remove those messages from unique_failures_df
unique_failures_df = unique_failures_df[~unique_failures_df['failure_message'].isin(no_existent_df['failure_message'])]

# Show counts, to get an idea of the balance and to check that the data split is being performed correctly
print(f"Original total in unique_failures_df: {original_count}")
print(f"Rows moved to no_existent_df: {len(no_existent_df)}")
print(f"Remaining in unique_failures_df: {len(unique_failures_df)}")


Original total in unique_failures_df: 109
Rows moved to no_existent_df: 52
Remaining in unique_failures_df: 57


In [4]:
# At first, the array was complete, but as I kept investigating, I started adding values to the list,
# and it gradually became smaller over time
unique_failures_df


,failure_message
0,"""JunkMail rejected - a227-71.mailgun.net [143.55.227.71]:44494 is in an RBL\non rbl.websitewelcome.com, see """
2,"30 Sorry, your message to ava@yahoo.com cannot be delivered. This mailbox is disabled (554.30)."
3,"30 Sorry, your message to bilallasuru@yahoo.com cannot be delivered. This mailbox is disabled (554.30)."
4,"30 Sorry, your message to cathiegriggs278@yahoo.com cannot be delivered. This mailbox is disabled (554.30)."
5,4.2.2 The recipient's inbox is out of storage space. Please direct the\n4.2.2 recipient to\n4.2.2 https://support.google.com/mail/?p=OverQuotaTemp a8-20020a05620a102800b0077bcea051c6si935804qkk.550 - gsmtp
6,4.2.2 The recipient's inbox is out of storage space. Please direct the\n4.2.2 recipient to\n4.2.2 https://support.google.com/mail/?p=OverQuotaTemp f16-20020ac859d0000000b00417dd658ebesi1059783qtf.686 - gsmtp
7,4.2.2 The recipient's inbox is out of storage space. Please direct the\n4.2.2 recipient to\n4.2.2 https://support.google.com/mail/?p=OverQuotaTemp l9-20020a05622a174900b0042377d1fcedsi1011145qtk.725 - gsmtp
8,4.4.5 The address <emailtosalesforce@12xzcb64ebbvu15vi5ijor1cbbutbj73ta5sk9c82tcxca8hc7.e-i5zqmac.na212.le.salesforce.com> is receiving mail at an excessive rate. Please resend your message at a later time.
9,5.0.0 <jiri.stulik@unob.cz>... Blocked: sender validity not confirmed [1].
37,5.2.1 The email account that you tried to reach is inactive. Learn more at\n5.2.1 https://support.google.com/mail/?p=DisabledUser f9-20020a05620a408900b00777279e49cbsi985299qko.165 - gsmtp


In [ ]:
# Here I was checking that the values were correct and providing a second opinion on whether the keywords were well chosen,
# just without using .head
no_existent_df

,failure_message
1,"1 Requested mail action aborted, mailbox not found"
10,5.1.1 <7063665190@vtext.com> recipient address rejected
11,5.1.1 <cloud3@digibazar.store>: Recipient address rejected: undeliverable address: host fr-int-smtpin8.hostinger.io[private/dovecot-lmtp] said: 550 5.1.1 <cloud3@digibazar.store> User doesn't exist: cloud3@digibazar.store (in reply to RCPT TO command)
12,"5.1.1 <info777@schmerz-freiheit.ch>: Email address could not be found, or was misspelled (G8)"
13,5.1.1 <jon.thorpe@mrcopper.com> Recipient not found. <https://www.secureserver.net/help/fix-rejected-email-with-a-bounce-error-40685?pl_id=1592&prog_id=domainspricedright#irbpl_id=1592&prog_id=domainspricedright#irb>
14,5.1.1 <ostovich888@me.com>: user does not exist
15,5.1.1 The email account that you tried to reach does not exist. Please try\n5.1.1 double-checking the recipient's email address for typos or\n5.1.1 unnecessary spaces. Learn more at\n5.1.1 https://support.google.com/mail/?p=NoSuchUser bq39-20020a05620a46a700b0077d876e4084si5769888qkb.548 - gsmtp
16,5.1.1 The email account that you tried to reach does not exist. Please try\n5.1.1 double-checking the recipient's email address for typos or\n5.1.1 unnecessary spaces. Learn more at\n5.1.1 https://support.google.com/mail/?p=NoSuchUser de42-20020a05620a372a00b00779df1bedbdsi1006625qkb.310 - gsmtp
17,5.1.1 The email account that you tried to reach does not exist. Please try\n5.1.1 double-checking the recipient's email address for typos or\n5.1.1 unnecessary spaces. Learn more at\n5.1.1 https://support.google.com/mail/?p=NoSuchUser e12-20020a056102310c00b004629594bcd9si691206vsh.100 - gsmtp
18,5.1.1 The email account that you tried to reach does not exist. Please try\n5.1.1 double-checking the recipient's email address for typos or\n5.1.1 unnecessary spaces. Learn more at\n5.1.1 https://support.google.com/mail/?p=NoSuchUser ea16-20020ad458b0000000b00679dfa08501si1623343qvb.421 - gsmtp


In [6]:

# Get the rows from the original DataFrame 'df' where 'failure_message' is in 'no_existent_df'
df_no_exist_rows = df[df['failure_message'].isin(no_existent_df['failure_message'])]

# Get only the 'recipient' column and apply .unique() to obtain unique values
df_no_exist_rows_recipient_unique = pd.DataFrame(df_no_exist_rows['recipient'].unique(), columns=["recipient"])

# Save the results to a CSV file
df_no_exist_rows_recipient_unique.to_csv('no_existent_emails.csv', index=False)
